### Autores:  
Blanco García, Gabriel: gabriel.blanco@cunef.edu  
Ferrín Meilán, Michelle: michelle.ferrin@cunef.edu

## Colegio Universitario de Estudios Financieros
### Máster en Data Science para Finanzas
Madrid, diciembre de 2020

# XGBoost

El XGBoost o extreme gradient boosting es un procedimiento para el ensamblado de algoritmos de Machine Learning. Sus siglas hacen referencia a eXtreme Gradient Boosting. Este método, al contrario que el bagging, no procesa en paralelo los modelos, es más complejo.

El procedimiento es secuencial. Se empieza con un modelo, que entrena con unos datos, y se evalua con un conjunto de test. Una vez evaluado el desempeño de ese primer modelo, se localizan aquellas observaciones en las que el modelo ha tenido más problemas y ha fallado. Estas observaciones tienen más probabilidad de aparecer en los datos para el siguiente modelo, y así sucesivamente. Lo que busca es pulir una y otra vez las observaciones que son difíciles de clasificar.  

Es un método potente y complejo, flexible en el sentido de que los modelos no tienen por qué ser del mismo tipo. El problema es que es caro en coste computacional,  y puede generar overfitting.

En este notebook construimos un XGBoost basado en árboles de decisión

In [17]:
# Manipulacion
import pandas as pd
import numpy as np

# Pipelines
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# Clasificador GradientBoosting
from sklearn.ensemble import GradientBoostingClassifier

# Evaluación preliminar del modelo
from sklearn import metrics 

# Cargar pipelines
import pickle  

# Guardar los modelos
%run ../src/operar_modelos.ipynb

# Omitir warnings
import warnings
warnings.filterwarnings('ignore')

Leemos los datos

In [13]:
X_train = pd.read_csv('../data/train/X_train.csv', engine='python')
y_train = pd.read_csv('../data/train/y_train.csv', engine='python')

X_test = pd.read_csv('../data/test/X_test.csv', engine='python')
y_test = pd.read_csv('../data/test/y_test.csv', engine='python')

Cargamos los pipelines

In [3]:
preprocesador = cargar_pipelines()

Creamos el modelo con preprocesado y clasificador GradientBoosting. No empleamos validación cruzada porque el tiempo de cómputo se extiende demasiado, pero sería la mejor manera de hacerlo.

In [19]:
XGBoost = Pipeline(steps=[
    ('preprocesador', preprocesador),
    
    ('clasificador', GradientBoostingClassifier(random_state=1234))]) # primero sin parámetros

Entrenamos el modelo

In [20]:
XGBoost.fit(X_train, y_train)

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('numericas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('escalador',
                                                                   StandardScaler())]),
                                                  Index(['loan_amnt', 'int_rate', 'annual_inc', 'dti', 'annual_inc_joint',
       'dti_joint', 'mort_acc'],
      dtype='object')),
                                                 ('categoricas',
                                                  Pipeline(steps=[('imputador',
                                                                   SimpleImputer(fill_value='perdido',
                                                                                 strategy='constant')),
                     

In [21]:
XGBoost.score(X_train, y_train) # 78.87% en train

0.7887160403008864

In [22]:
XGBoost.score(X_test, y_test) # 78.81% en test

0.7881521868475427

La precisión no varía sustancialmente entre train y test, así que se descarta el overfitting

Guardamos el modelo y pasamos a la selección de modelos

In [ ]:
guardar_modelo(XGBoost, '../models/trained_models/XGBoost.sav')